In [221]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [222]:
class PCA:
    
    def fit(self, X):
        
        mean_vec = np.mean(X, axis=0)
        cov_mat = (X - mean_vec).T.dot((X - mean_vec)) / (X.shape[0]-1)
        eig_vals, eig_vecs = np.linalg.eig(cov_mat)

        # Make a list of (eigenvalue, eigenvector) tuples
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

        # Sort the (eigenvalue, eigenvector) tuples from high to low
        eig_pairs.sort(reverse=True)

        # statically take two principal components 
        eig_pairs = eig_pairs[:2]

        # make projection matrix from principal components
        self.components = np.hstack(map(lambda eig_pair: eig_pair[1].reshape(X.shape[1],1) , eig_pairs))
        
    def project(self, X):
        """ Project input data onto the space of the principal components """
        return np.dot(X, self.components)

In [246]:
# generate dataset
X, Y = datasets.make_regression(n_samples=100, n_features=2, n_informative=2, noise=30, bias=0)
X = np.stack([X[:,0], X[:,1],Y], axis=1)
X = X / np.std(X, axis=0)

# train PCA
pca = PCA()
pca.fit(X)

""" plot projection plane section """

# grid for surface
x_axis = np.linspace(min(X[:,0]), max(X[:,0]), 10)
y_axis = np.linspace(min(X[:,1]), max(X[:,1]), 10)
X_grid, Y_grid = np.meshgrid(x_axis, y_axis)

# calc normal vector to plane
normal = np.cross(pca.components[:,0] , pca.components[:,1])

# the normal is (a, b, c) 
# and the plane equation is a*x + b*y + c*z = 0
# solve for Z
Z = ( -1 * normal[0] * X_grid - 1 * normal[1] * Y_grid ) / normal[2]

# plot points with plotly
trace1 = go.Scatter3d(
    x=X[:,0],
    y=X[:,1],
    z=X[:,2],
    mode='markers',
    marker=dict(size=4, color='red', line=dict(color='black', width=0.5), opacity=0.8)
)

# plot plane with plotly
trace2 = go.Surface(z=Z, x=X_grid, y=Y_grid, colorscale='RdBu', opacity=0.6)

# Package the trace dictionary into a data object
data = [trace1, trace2]

# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = go.Layout(
    title='PCA visualization', # set plot title
    scene=go.layout.Scene(  # axes are part of a 'scene' in 3d plots
        xaxis=go.layout.scene.XAxis(axis), # set x-axis style
        yaxis=go.layout.scene.YAxis(axis), # set y-axis style
        zaxis=go.layout.scene.ZAxis(axis)),  # set z-axis style
    autosize=False,
    width=800,
    height=800,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=40
    ),
)

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send to Plotly and show in notebook
py.iplot(fig, filename='test1')

/anaconda/lib/python3.5/site-packages/plotly/graph_objs/_deprecations.py:396: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~fgranqvist/0 or inside your plot.ly account where it is named 'test1'
